![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/4.11.Deidentification_NER_Profiling_Pipeline.ipynb)

# PHI Detection Profiling Pipeline

This pipeline is designed for profiling and benchmarking various de-identification models applied to clinical texts. It integrates multiple NER models and rule-based components that are commonly used for detecting and anonymizing protected health information (PHI). The pipeline includes models trained with embeddings_clinical, zero-shot NER models, regex matchers, text matchers, and contextual parsers. By consolidating these diverse approaches, it allows comprehensive evaluation and comparison of different de-identification strategies across clinical datasets.

# Setup

In [ ]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.4 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

In [3]:
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql import SparkSession

import sparknlp_jsl
import sparknlp

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *

params = {"spark.driver.memory":"16G",
"spark.kryoserializer.buffer.max":"2000M",
"spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 6.0.0
Spark NLP_JSL Version : 6.0.0


# Pipeline

In [13]:
from sparknlp.pretrained import PretrainedPipeline

phi_detection_pipeline = PretrainedPipeline('ner_profiling_deidentification', 'en', 'clinical/models')
phi_detection_pipeline.model.stages

[DocumentAssembler_787f0c7ce81a,
 SentenceDetectorDLModel_6bafc4746ea5,
 REGEX_TOKENIZER_193178adde3e,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 MedicalNerModel_8aa43467dabc,
 NerConverter_94c565b83311,
 MedicalNerModel_2dd7df45612b,
 NerConverter_e84629089bd2,
 MedicalNerModel_8bdd6c4bd644,
 NerConverter_5e606d0f25b3,
 MedicalNerModel_d4076802b007,
 NerConverter_f2bc948b1ee8,
 MedicalNerModel_b611869af9b6,
 NerConverter_63ad42dab614,
 MedicalNerModel_9d4a08b1c03d,
 NerConverter_d4ff1b9d813e,
 MedicalNerModel_8f4e637b302a,
 NerConverter_318269814f5e,
 MedicalNerModel_cb1ebd97c79c,
 NerConverter_3b7bbc78c52a,
 MedicalNerModel_ada39ac0d359,
 NerConverter_fb0ed6eac19f,
 MedicalNerModel_15975bb9777e,
 NerConverter_449a16de318d,
 MedicalNerModel_80ef11d39c77,
 NerConverter_131d01d822f4,
 MedicalNerModel_132c5bcaaa62,
 NerConverter_f559790d7820,
 MedicalNerModel_d92d47622e85,
 NerConverter_16421fbcadbc,
 MedicalNerModel_32184c1db80b,
 NerConverter_ae1044c220ac,
 MedicalNerModel_5d9ebaf6a7c9,
 Ne

## Prediction

You can run the sample text through the pipeline with a single line of code to get predictions from all PHI detection models available in the Healthcare NLP library.

In [20]:
text= """Name : Hendrickson, Ora, Record date: 2093-01-13, Age: 25, # 719435. Dr. John Green, ID: 1231511863, IP 203.120.223.13. He is a 60-year-old male was admitted to the Day Hospital for cystectomy on 01/13/93. Patient's VIN : 1HGBH41JXMN109286, SSN #333-44-6666, Driver's license no:A334455B. Phone (302) 786-5227, 0295 Keats Street, San Francisco."""

result = phi_detection_pipeline.fullAnnotate(text)

## Evaluation

In [156]:
import pandas as pd
import plotly.graph_objects as go

class NERResultViewer:
    """
    A utility class to visualize NER (Named Entity Recognition) model outputs
    in different formats: chunks, token-level predictions, tuple results, and label distribution plots.
    """

    def __init__(self, light_result):
        """
        Initialize the class with the light_result dictionary from Spark NLP or similar output.
        """
        self.light_result = light_result

    def build_chunk_dataframe(self, results):
        """
        Helper function that builds a DataFrame from chunk results.
        """
        data = {
            'sentence': [r.metadata.get('sentence', '') for r in results],
            'begin': [r.begin for r in results],
            'end': [r.end for r in results],
            'chunks': [r.result for r in results],
            'entity': [r.metadata.get('entity') if r.metadata.get('entity') else r.metadata.get('field', '') for r in results],
            'confidence': [r.metadata.get('confidence', '') for r in results]
        }
        return pd.DataFrame(data)

    def show_chunk_results(self):
        """
        Prints chunk-level NER results as DataFrames for each model.
        """
        for key in self.light_result.keys():
            if key in ['sentence', 'token'] or "_chunks" not in key:
                continue

            model_name = "_".join(key.split("_")[:-1])
            results = self.light_result[key]

            print(f"\n{'*' * 20} {model_name} Model Results {'*' * 20}")

            if len(results) == 0:
                print("No Result For This Model")
                continue

            df = self.build_chunk_dataframe(results)
            display(df)

    def get_token_results(self):
        """
        Generate a DataFrame showing token-level predictions for all models.
        """

        # Extract base token data
        token_data = self.light_result["token"]
        df = pd.DataFrame({
            'sentence': [t.metadata["sentence"] for t in token_data],
            'begin': [t.begin for t in token_data],
            'end': [t.end for t in token_data],
            'token': [t.result for t in token_data]
        })

        # Add neural NER model predictions
        for key, model_output in self.light_result.items():
            if key in ["sentence", "token"] or "_chunks" in key or "ner" not in key:
                continue

            labels = [entry.result for entry in model_output]
            df[key] = labels

        # Add rule-based chunk predictions (like age_parser_chunks)
        for key, chunk_annotations in self.light_result.items():
            if key in ["sentence", "token"] or "ner" in key:
                continue

            chunk_df = self.build_chunk_dataframe(chunk_annotations)

            # Build an index-efficient token -> label mapping
            labels = []
            for _, token_row in df.iterrows():
                matched_entity = "O"
                for _, chunk_row in chunk_df.iterrows():
                    if (chunk_row.begin <= token_row.begin < chunk_row.end) or (token_row.begin <= chunk_row.begin < token_row.end):
                        matched_entity = chunk_row.entity
                        break
                labels.append(matched_entity)

            column_name = "_".join(key.split("_")[:-1])
            df[column_name] = labels

        return df


    def show_results_as_tuples(self):
        """
        Print the NER token predictions as (token, label) tuples for each model.
        """
        tokens = [j.result for j in self.light_result["token"]]

        for key in self.light_result.keys():
            if key == 'sentence' or "_chunks" in key:
                continue

            results = self.light_result[key]

            print(f"\n{'*'*20} {key} Model Results {'*'*20}")

            if len(results) == 0:
                print("No Result For This Model")
                continue

            labels = [r.result for r in results]
            paired = list(zip(tokens, labels))
            print(paired)

    def plot_entity_counts(self):
        """
        Plot a horizontal bar chart of entity counts (excluding "O") per model using Plotly.
        """
        # Get token-level result DataFrame
        result_df = self.get_token_results()

        # Extract only label columns (excluding token info)
        label_result_df = result_df[result_df.columns[4:]].copy()

        # Count how many non-'O' labels exist for each model
        label_count_list = [label_result_df[label_result_df[col] != "O"].shape[0] for col in label_result_df.columns]

        # Create bar plot
        fig = go.Figure(go.Bar(
            x=label_count_list,
            y=label_result_df.columns,
            orientation='h'
        ))

        fig.update_layout(
            autosize=False,
            width=1500,
            height=1500,
            margin=dict(l=50, r=50, b=100, t=100, pad=4),
            paper_bgcolor="LightSteelBlue",
            title={'text': "Counts of Labelled Entities", 'y':0.98, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
            titlefont=dict(size=30),
            yaxis=dict(title_text="Clinical NER Models", titlefont=dict(size=30)),
        )

        fig.show()

In [157]:
# Instantiate the class with your model output
viewer = NERResultViewer(result[0])  # Use light_result if not in a list

### Chunk Based Predictions

In [158]:
# Show chunk-level results
viewer.show_chunk_results()


******************** ner_deid_aipii Model Results ********************


,sentence,begin,end,chunks,entity,confidence
0,0,7,17,Hendrickson,NAME,0.9032
1,0,38,47,2093-01-13,SSN,0.8288
2,1,73,82,John Green,STREET,0.5083
3,1,89,98,1231511863,IDNUM,0.9761
4,1,104,117,203.120.223.13,SSN,0.8333
5,4,295,308,(302) 786-5227,PHONE,0.87525004
6,4,311,327,0295 Keats Street,STREET,0.6360667
7,4,330,342,San Francisco,CITY,0.72644997



******************** state_matcher Model Results ********************
No Result For This Model

******************** ssn_parser Model Results ********************


,sentence,begin,end,chunks,entity,confidence
0,3,246,256,333-44-6666,SSN,0.73



******************** phone_parser Model Results ********************


...


******************** country_matcher Model Results ********************
No Result For This Model

******************** date_matcher Model Results ********************


,sentence,begin,end,chunks,entity,confidence
0,0,38,47,2093-01-13,DATE,None
1,2,196,203,01/13/93,DATE,None
2,3,247,256,33-44-6666,DATE,None



******************** ner_deid_subentity_augmented_docwise Model Results ********************


,sentence,begin,end,chunks,entity,confidence
0,0,7,22,"Hendrickson, Ora",PATIENT,0.9607666
1,0,38,47,2093-01-13,DATE,1.0
2,0,55,56,25,AGE,0.9904
3,0,61,66,719435,PHONE,0.9801
4,1,73,82,John Green,DOCTOR,0.99845004
5,1,89,98,1231511863,IDNUM,0.6383
6,1,104,117,203.120.223.13,PHONE,0.7589
7,2,128,138,60-year-old,AGE,1.0
8,2,165,176,Day Hospital,HOSPITAL,0.86335
9,2,196,203,01/13/93,DATE,1.0



******************** date_of_birth_parser Model Results ********************
No Result For This Model


### Specific Model Results

You can get the specific model results by using `build_chunk_dataframe` method.

In [172]:
result[0]["ip_matcher_chunks"]

[Annotation(chunk, 104, 117, 203.120.223.13, {'entity': 'IP', 'ner_source': 'ip_matcher_chunks', 'chunk': '0', 'sentence': '0'}, [])]

In [173]:
viewer.build_chunk_dataframe(result[0]["ip_matcher_chunks"])

,sentence,begin,end,chunks,entity,confidence
0,0,104,117,203.120.223.13,IP,None


### Token Based Predictions

In [159]:
# Get token-level DataFrame
df = viewer.get_token_results()
df

,sentence,begin,end,token,ner_deid_augmented,ner_deid_subentity_augmented_i2b2,ner_deidentify_dl,ner_deid_subentity_augmented_langtest,ner_deid_synthetic,ner_deid_aipii,ner_deid_large_langtest,ner_deid_large,ner_deid_subentity_docwise,ner_deid_subentity_augmented_v2,ner_deid_subentity_augmented_docwise,zeroshot_ner_deid_subentity_merged_medium,ner_deid_sd_large,ner_deid_enriched_langtest,ner_deid_sd,ner_deid_generic_augmented,ner_deid_enriched,ner_deid_generic_augmented_langtest,ner_deid_subentity_augmented,ner_deid_generic_docwise,ner_deid_generic_augmented_allUpperCased_langtest,state_matcher,ssn_parser,phone_parser,zip_matcher,ip_matcher,medical_record_parser,date_of_death_parser,drug_parser,account_parser,plate_parser,url_matcher,email_matcher,zip_parser,license_parser,phone_matcher,dln_parser,vin_parser,age_parser,country_matcher,date_matcher,date_of_birth_parser
0,0,0,3,Name,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
1,0,5,5,:,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
2,0,7,17,Hendrickson,B-NAME,B-PATIENT,B-PATIENT,B-PATIENT,B-NAME,B-NAME,B-NAME,B-NAME,B-PATIENT,B-PATIENT,B-PATIENT,B-PATIENT,B-NAME,B-PATIENT,B-NAME,B-NAME,B-PATIENT,B-NAME,B-PATIENT,B-NAME,B-NAME,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
3,0,18,18,",",I-NAME,I-PATIENT,I-PATIENT,I-PATIENT,I-NAME,O,I-NAME,I-NAME,I-PATIENT,I-PATIENT,I-PATIENT,I-PATIENT,I-NAME,I-PATIENT,I-NAME,I-NAME,I-PATIENT,I-NAME,I-PATIENT,I-NAME,I-NAME,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
4,0,20,22,Ora,I-NAME,I-PATIENT,I-PATIENT,I-PATIENT,I-NAME,O,I-NAME,I-NAME,I-PATIENT,I-PATIENT,I-PATIENT,I-PATIENT,I-NAME,I-PATIENT,I-NAME,I-NAME,I-PATIENT,I-NAME,I-PATIENT,I-NAME,I-NAME,O,O,O,O,O,O,O,Drug,O,O,O,O,O,O,O,O,O,O,O,O,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,4,322,327,Street,I-LOCATION,I-STREET,I-STREET,I-STREET,I-LOCATION,I-STREET,I-LOCATION,I-LOCATION,I-STREET,I-STREET,I-LOCATION,I-STREET,I-LOCATION,I-STREET,I-LOCATION,I-LOCATION,I-STREET,I-LOCATION,I-STREET,I-LOCATION,I-LOCATION,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
67,4,328,328,",",O,O,O,O,O,O,O,O,O,O,I-LOCATION,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
68,4,330,332,San,B-LOCATION,B-CITY,B-CITY,B-CITY,B-LOCATION,B-CITY,B-LOCATION,B-LOCATION,B-CITY,B-STATE,I-LOCATION,B-CITY,B-LOCATION,B-CITY,B-LOCATION,B-LOCATION,B-CITY,B-LOCATION,B-STATE,B-LOCATION,B-LOCATION,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
69,4,334,342,Francisco,I-LOCATION,I-CITY,I-CITY,I-CITY,I-LOCATION,I-CITY,I-LOCATION,I-LOCATION,I-CITY,I-STATE,I-LOCATION,I-CITY,I-LOCATION,I-CITY,I-LOCATION,I-LOCATION,I-CITY,I-LOCATION,I-STATE,I-LOCATION,I-LOCATION,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O


### Review the Results In the Form of Token-Label Tuples

In [163]:
# Show (token, label) tuples
viewer.show_results_as_tuples()


******************** ner_deid_augmented Model Results ********************
[('Name', 'O'), (':', 'O'), ('Hendrickson', 'B-NAME'), (',', 'I-NAME'), ('Ora', 'I-NAME'), (',', 'O'), ('Record', 'O'), ('date', 'O'), (':', 'O'), ('2093-01-13', 'B-DATE'), (',', 'O'), ('Age', 'O'), (':', 'O'), ('25', 'B-AGE'), (',', 'O'), ('#', 'O'), ('719435', 'B-CONTACT'), ('.', 'O'), ('Dr', 'O'), ('.', 'O'), ('John', 'B-NAME'), ('Green', 'I-NAME'), (',', 'O'), ('ID', 'O'), (':', 'O'), ('1231511863', 'B-ID'), (',', 'O'), ('IP', 'B-CONTACT'), ('203.120.223.13', 'I-CONTACT'), ('.', 'O'), ('He', 'O'), ('is', 'O'), ('a', 'O'), ('60-year-old', 'O'), ('male', 'O'), ('was', 'O'), ('admitted', 'O'), ('to', 'O'), ('the', 'O'), ('Day', 'B-LOCATION'), ('Hospital', 'I-LOCATION'), ('for', 'O'), ('cystectomy', 'O'), ('on', 'O'), ('01/13/93', 'B-DATE'), ('.', 'O'), ("Patient's", 'O'), ('VIN', 'O'), (':', 'O'), ('1HGBH41JXMN109286', 'B-ID'), (',', 'O'), ('SSN', 'O'), ('#333-44-6666', 'O'), (',', 'O'), ("Driver's", 'O'), ('l

### Comparison of Model Prediction Frequencies

In this bar chart, you can see how many PHI **tokens** were captured by each model.

In [164]:
# Plot PHI TOKEN counts
viewer.plot_entity_counts()